# MLB Database Design

We will be working with a file of Major League Baseball games from [Retrosheet](https://www.retrosheet.org). Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. The main file we will be working from **game_log.csv**, has been produced by combining 127 separate CSV files from retrosheet, and has been pre-cleaned to remove some inconsistencies. The game log has hundreds of data points on each game which we will normalize into several separate tables using SQL, providing a robust database of game-level statistics.

In addition to the main file, we have also included three 'helper' files, also sourced from Retrosheet:

- park_codes.csv
- person_codes.csv
- team_codes.csv

These three helper files in some cases contain extra data, but will also make things easier as they will form the basis for three of our normalized tables.

In [2]:
import pandas as pd
import sqlite3

In [3]:
game_log = pd.read_csv('game_log.csv', low_memory=False)
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

In [4]:
game_log.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,NaN,2,RC1,NaN,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,NaN,3,CH1,NaN,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,NaN,2,TRO,NaN,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [5]:
park_codes.head()

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [6]:
person_codes.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


In [7]:
team_codes.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


In [8]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


Of the four files, the main game_log file contains the statistics for each game, including information on the people, teams, and venue involved.

Columns 4 and 7 contain the home and away team IDs which match with the team_id column of the team_codes file. 

Column 17 contains the Park ID which matches with the park_id column of the park_codes file. 

Columns 78 - 159 contain information on the people involved with the game. This includes, umpires, managers, coaches, and players. Each with name and ID which will match with the ID column of the person_codes file. 

The defensive position for each of the players here represents the order in which they appeared in the batting lineup (in range of 1-9)

In [9]:
park_codes.league.value_counts()

NL    88
AL    47
AA    33
UA     9
FL     7
PL     2
Name: league, dtype: int64

The park league codes correspond to the specific league the park belongs to. There are 30 teams that play in the **National League (NL)** and **American League (AL)**, 15 in each. 

Several other early defunct baseball leagues are officially considered major leagues, and their statistics and records are included with those of the two current major leagues. These include:

- **American Association (AA)** (1882-1891)
- **Union Association (UA)** (1884)
- **Players' League (PL)** (1890)
- **Federal League (FL)** (1914–1915)

Both the UA and AA are considered major leagues by many baseball researchers because of the perceived high caliber of play and the number of star players featured. 

## Importing the data

Our next task is to import the data into SQLite. There are three key ways to import data into a SQLite database:

1. Using the Python SQLite library
2. Using Pandas
3. From the SQLite shell

In this instance, we will proceed with the pandas method as the dataset is sufficiently small to be held in memory, and for ease of use. 

In [12]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql_query(q, conn)
    
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [ ]:
conn = sqlite3.connect('mlb.db')
game_log.to_sql('game_log', conn, index=False)
park_codes.to_sql('park_codes', conn, index=False)
person_codes.to_sql('person_codes', conn, index=False)
team_codes.to_sql('team_codes', conn, index=False)

To insert data into a normalised database, we'll need a single column that can be used as a primary key. The game log file does not have a single column that can be used as a primary key to uniquely identify each game. There are three ways that we could handle this:

1. Make a compound primary key, such as a primary key of the date, h_name, and number_of_game columns.
2. Insert an integer primary key, eg where the first row is 1, the second row is 2, etc.
3. Insert a new column using a custom format.

Exploring the Retrosheet site, we can find this [data dictionary](https://www.retrosheet.org/eventfile.htm) for their event files, which list every event within each game. This includes the following description:

*id: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, ATL198304080 should be read as follows.*

1. The first three characters identify the home team (the Braves). 
2. The next four are the year (1983). 
3. The next four are the month (April) using the standard numeric notation, 04, followed by the day (08). 
4. The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header 

*The id record starts the description of a game thus ending the description of the preceding game in the file.*

In [11]:
c = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''
run_command(c)

c = '''
UPDATE game_log
SET game_id = h_name || date || number_of_game;
'''
run_command(c)

In [12]:
q = 'SELECT * FROM game_log LIMIT 5'
run_query(q)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,...,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105040
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,...,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y,WS3187105050
2,18710506,0,Sat,CL1,None,2,RC1,None,1,12,...,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,None,Y,RC1187105060
3,18710508,0,Mon,CL1,None,3,CH1,None,1,12,...,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,None,Y,CH1187105080
4,18710509,0,Tue,BS1,None,2,TRO,None,1,9,...,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y,TRO187105090


## Data normalisation

We're now going to look for specific opportunities to normalize our data by reducing repetition. Here are three examples we can find and remove:

#### Repition in columns
The following are columns in the game_log table:

v_player_1_id, v_player_1_name, v_player_1_def_pos, v_player_2_id, v_player_2_name, v_player_2_def_pos

We have three columns that relate to one player, followed by three columns that relate to another player. We could restructure our data to remove this repetition - we would need to add an extra column to include the data that was previously only contained in the name of the column:

id, name, def_pos, off_pos

#### Non-primary key columns should be attributes of the primary key
The primary key of our game log is our game_id, but the players name are not attributes of a game, but of the player id. If the only data we had was the game log, we would remove this column and create a new table that had the names of each player. As it happens, our person_codes table already has a list of our player IDs and names, so we can remove these without the need for creating a new table first.

#### Redundant data
Lastly, we want to eliminate any redundant data - that is, columns where the data is available elsewhere. A good example of this can be found in the park_codes table. 

The start and end columns show the first and last games played at the park, however we will be able to derive this information by looking at the park information for each game. Similarly, the league information is going to be available elsewhere in our database.

## Database design

Now that we have gone through a few scenarios of data normalisation to keep in mind, the following schema will guide the rest of the creation. 

![Database schema](https://s3.amazonaws.com/dq-content/193/mlb_schema.svg)

## Creating tables

We'll start by creating the tables that don't contain any foreign key relations. It's important to start with these tables, as other tables will have relations to these tables, and so these tables will need to exist first. These include:

- **person**
    - Each of the 'debut' columns have been omitted, as the data will be able to be found from other tables.
    - Since the game log file has no data on coaches, we made the decision to not include this data.
- **park**
    - The start, end, and league columns contain data that is found in the main game log and can be removed.
- **league**
    - Because some of the older leagues are not well known, we will create a table to store league names.
- **appearance_type**
    - Our appearance table will include data on players with positions, umpires, managers, and awards (like winning pitcher). This table will store information on what different types of appearances are available.

In [13]:
# creating table for person info
c = '''
CREATE TABLE person (
person_id TEXT PRIMARY KEY,
first_name TEXT, 
last_name TEXT);
'''
run_command(c)

# copying data from person_codes table
c = '''
INSERT INTO person
SELECT id, first, last FROM person_codes;
'''
run_command(c)

# querying to test table creation was successful
q = 'SELECT * FROM person LIMIT 5'
run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [14]:
# creating table for park info
c = '''
CREATE TABLE park (
park_id TEXT PRIMARY KEY,
name TEXT, 
nickname TEXT,
city TEXT, 
state TEXT,
notes TEST);
'''
run_command(c)

# copying data from park_codes table
c = '''
INSERT INTO park
SELECT park_id, name, aka, city, state, notes FROM park_codes;
'''
run_command(c)

# querying to test table creation was successful
q = 'SELECT * FROM park LIMIT 5'
run_query(q)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [15]:
# creating table for league info
c = '''
CREATE TABLE league (
league_id TEXT PRIMARY KEY,
name TEXT
);
'''
run_command(c)

# manually adding data for leagues
c = '''
INSERT INTO league
VALUES
("NL", "National League"), 
("AL", "American League"),
("AA", "American Association"),
("UA", "Union Association"), 
("PL", "Players League"), 
("FL", "Federal League");
'''
run_command(c)

# querying to test table creation was successful
q = 'SELECT * FROM league'
run_query(q)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,UA,Union Association
4,PL,Players League
5,FL,Federal League


In [11]:
appearance_type = pd.read_csv('appearance_type.csv')
appearance_type.head()

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [14]:
# creating table for appearance type info
c = '''
CREATE TABLE IF NOT EXISTS appearance_type (
appearance_type_id TEXT PRIMARY KEY,
name TEXT,
category TEXT);
'''

# uploading dataframe data to apearance type SQL table
with sqlite3.connect('mlb.db') as conn:
    appearance_type.to_sql('appearance_type', conn, index=False, if_exists='append')
    
# querying to test table creation was successful
q = 'SELECT * FROM appearance_type LIMIT 5'

run_command(c)
run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


Now that we have added all of the tables that don't have foreign key relationships, lets add the next two tables. The game and team tables need to exist before our two appearance tables are created.

- **team**
    - The start, end, and sequence columns can be derived from the game level data.
- **game**
    - We have chosen to include all columns for the game log that don't refer to one specific team or player, instead putting those in two appearance tables.
    - We have removed the column with the day of the week, as this can be derived from the date.
    - We have changed the day_night column to day, with the intention of making this a boolean column.

In [24]:
# starting by checking for duplicate team_id values
q = '''
WITH duplicates AS (
    SELECT *
    FROM (
        SELECT team_id, count(*) entries
        FROM team_codes
        GROUP BY 1
        )
    WHERE entries > 1
)

SELECT * FROM team_codes
WHERE team_id = (SELECT team_id FROM duplicates)
'''
run_query(q)

,team_id,league,start,end,city,nickname,franch_id,seq
0,MIL,AL,1970,1997,Milwaukee,Brewers,SE1,2
1,MIL,NL,1998,0,Milwaukee,Brewers,SE1,3


In [27]:
# deleting the older duplicate row
c = '''
DELETE FROM team_codes
WHERE 
    team_id = 'MIL'
AND
    league = 'AL'
'''
run_command(c)

In [30]:
# confirm the row has been removed
q = '''
SELECT *
FROM (
    SELECT team_id, count(*) entries
    FROM team_codes
    GROUP BY 1
    )
WHERE entries > 1
'''
run_query(q)

,team_id,entries


In [31]:
# creating team table with foreign key to league table
c = '''
CREATE TABLE team (
team_id TEXT PRIMARY KEY,
league_id TEXT,
city TEXT,
nickname TEXT, 
franch_id TEXT,
FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''
run_command(c)

# inserting data from team_codes table
c = '''
INSERT INTO team
SELECT DISTINCT team_id, league, city, nickname, franch_id FROM team_codes
'''
run_command(c)

# querying table to test successful creation
q = 'SELECT * FROM team LIMIT 5'
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [53]:
# creating game table
c = '''
CREATE TABLE game (
game_id TEXT PRIMARY KEY,
date NUMERIC, 
number_of_game NUMERIC,
park_id TEXT,
length_outs NUMERIC,
day NUMERIC,
completion TEXT, 
forfeit TEXT, 
protest TEXT, 
attenance NUMERIC, 
length_minutes NUMERIC, 
additional_info TEXT,
acquisition_info TEXT,
FOREIGN KEY (park_id) REFERENCES park(park_id)
);
'''
run_command(c)

# inserting data from game_log table, and converting day_night to boolean
c = '''
INSERT INTO game
SELECT 
    game_id, 
    date, 
    number_of_game, 
    park_id, 
    length_outs, 
    CASE 
        WHEN day_night = 'D' THEN 1
        WHEN day_night = 'N' THEN 0
        ELSE NULL
    END AS day,
    completion, 
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log
'''
run_command(c)

# testing table creation
q = 'SELECT * FROM game LIMIT 5'
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attenance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


Next we will create the team_appearance table. The team_appearance table has a compound primary key composed of the team name and the game ID. In addition, a boolean column home is used to differentiate between the home and the away team. The rest of the columns are scores or statistics that in our original game log are repeated for each of the home and away teams.

In [71]:
q = '''
SELECT sql FROM sqlite_master
WHERE name = "game_log" AND type = "table";
'''
result = run_query(q)
cols = result['sql'].iloc[0].replace('\n', '').replace('"', '').split('(')[1][:-1]
cols = cols.split(',  ')
cols

['date INTEGER',
 'number_of_game INTEGER',
 'day_of_week TEXT',
 'v_name TEXT',
 'v_league TEXT',
 'v_game_number INTEGER',
 'h_name TEXT',
 'h_league TEXT',
 'h_game_number INTEGER',
 'v_score INTEGER',
 'h_score INTEGER',
 'length_outs REAL',
 'day_night TEXT',
 'completion TEXT',
 'forefeit TEXT',
 'protest TEXT',
 'park_id TEXT',
 'attendance REAL',
 'length_minutes REAL',
 'v_line_score TEXT',
 'h_line_score TEXT',
 'v_at_bats REAL',
 'v_hits REAL',
 'v_doubles REAL',
 'v_triples REAL',
 'v_homeruns REAL',
 'v_rbi REAL',
 'v_sacrifice_hits REAL',
 'v_sacrifice_flies REAL',
 'v_hit_by_pitch REAL',
 'v_walks REAL',
 'v_intentional_walks REAL',
 'v_strikeouts REAL',
 'v_stolen_bases REAL',
 'v_caught_stealing REAL',
 'v_grounded_into_double REAL',
 'v_first_catcher_interference REAL',
 'v_left_on_base REAL',
 'v_pitchers_used REAL',
 'v_individual_earned_runs REAL',
 'v_team_earned_runs REAL',
 'v_wild_pitches REAL',
 'v_balks REAL',
 'v_putouts REAL',
 'v_assists REAL',
 'v_errors 

In [23]:
# creating team appearance table
c1 = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT, 
    game_id TEXT, 
    home BOOLEAN, 
    league_id TEXT, 
    score INTEGER, 
    line_score TEXT,
    at_bats REAL, 
    hits REAL, 
    doubles INTEGER,
    triples INTEGER, 
    homeruns INTEGER, 
    rbi INTEGER, 
    sacrifice_hits INTEGER, 
    sacrifice_flies INTEGER, 
    hit_by_pitch INTEGER, 
    walks INTEGER, 
    intentional_walks INTEGER, 
    strikeouts INTEGER, 
    stolen_bases INTEGER, 
    caught_stealing INTEGER, 
    grounded_into_double INTEGER, 
    first_catcher_interference INTEGER, 
    left_on_base INTEGER, 
    pitchers_used INTEGER, 
    individual_earned_runs INTEGER, 
    team_earned_runs INTEGER,
    wild_pitches INTEGER, 
    balks INTEGER, 
    putouts INTEGER, 
    assists INTEGER,
    errors INTEGER, 
    passed_balls INTEGER, 
    double_plays INTEGER, 
    triple_plays INTEGER,    
    PRIMARY KEY (team_id, game_id), 
    FOREIGN KEY (team_id) REFERENCES team(team_id), 
    FOREIGN KEY (game_id) REFERENCES game(game_id), 
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);'''

# inserting home and away team stats by querying seperately and unioning
c2 = '''
INSERT INTO team_appearance
    SELECT
        h_name,
        game_id, 
        1 AS home, 
        h_league, 
        h_score, 
        h_line_score, 
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log
        
UNION

    SELECT
        v_name,
        game_id, 
        0 AS home, 
        v_league, 
        v_score, 
        v_line_score, 
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log;
'''

# testing table creation
q = '''
SELECT * FROM team_appearance
WHERE game_id = 'FW1187105040'
   OR game_id = 'WAS201610020'
ORDER By game_id, home;
'''

run_command(c1)
run_command(c2)
run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,...,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,CL1,FW1187105040,0,None,0,000000000,30.0,4.0,1,0,...,1,1,0,0,27,9,0,3,0,0
1,FW1,FW1187105040,1,None,2,010010000,31.0,4.0,1,0,...,0,0,0,0,27,3,3,1,1,0
2,MIA,WAS201610020,0,NL,7,000230020,38.0,14.0,1,1,...,10,10,1,0,24,11,0,0,1,0
3,WAS,WAS201610020,1,NL,10,03023002x,30.0,10.0,2,0,...,7,7,1,0,27,11,0,0,1,0


The final table we need to create is person_appearance.

The person_appearance table will be used to store information on appearances in games by managers, players, and umpires as detailed in the appearance_type table.

In [25]:
c0 = 'DROP TABLE IF EXISTS person_appearance'

# final person appearance table creation
c1 = '''
CREATE TABLE IF NOT EXISTS person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT, 
    team_id TEXT, 
    game_id TEXT, 
    appearance_type_id TEXT, 
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id), 
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
'''

# inserting data for managers, umpires, pitchers, and awards.
c2 = '''
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
'''

# command template for inserting player appearances
template = '''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
'''

run_command(c0)
run_command(c1)
run_command(c2)

# uses the command template to loop through home & away players of all positions
for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [27]:
# query all of the person appearance for a specific game
q = '''
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = 'WAS201610020'
ORDER BY team_id, appearance_type_id
'''

run_query(q)

,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1591102,porta901,None,WAS201610020,U1B,First Base,umpire
1,1591101,onorb901,None,WAS201610020,U2B,Second Base,umpire
2,1591100,kellj901,None,WAS201610020,U3B,Third Base,umpire
3,1591103,tumpj901,None,WAS201610020,UHP,Home Plate,umpire
4,1591104,brica001,MIA,WAS201610020,ALP,Losing Pitcher,award
5,6706897,koeht001,MIA,WAS201610020,D1,Pitcher,defense
6,4735171,telit001,MIA,WAS201610020,D2,Catcher,defense
7,5580199,bourj002,MIA,WAS201610020,D3,1st Base,defense
8,4453495,gordd002,MIA,WAS201610020,D4,2nd Base,defense
9,5016847,pradm001,MIA,WAS201610020,D5,3rd Base,defense


## Removing original tables

Now that we all of the normalised tables have been created and filled with data, we can remove the original tables we used to import data from the CSVs.

In [28]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,person,table
5,park,table
6,league,table
7,appearance_type,table
8,appearance_type_import,table
9,team,table


In [32]:
tables = [
    "game_log",
    "park_codes",
    "team_codes",
    "person_codes"
]

for t in tables:
    c = '''
    DROP TABLE IF EXISTS {}
    '''.format(t)
    
    run_command(c)

show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table


## Next steps...

Here are some potential next steps:

- Transform the the dates into a SQLite compatible format.
- Extract the line scores into innings level data in a new table.
- Create views to make querying stats easier, eg:
    - Season level stats.
    - All time records.
- Supplement the database using new data, for instance:
    - Add data from retrosheet game logs for years after 2016.
    - Source and add missing pitcher information.
    - Add player level per-game stats.
    - Source and include base coach data.